In [1]:
import qbraid
from qbraid.devices._utils import SUPPORTED_VENDORS
from braket.devices import LocalSimulator
from braket.aws import AwsDevice
from braket.circuits import Circuit

In [2]:
def device_wrapper_input(vendor):
    input_list = []
    for provider in SUPPORTED_VENDORS[vendor]:
        for device in SUPPORTED_VENDORS[vendor][provider]:
            if provider != "D-Wave":
                data = (device, provider, vendor)
                input_list.append(data)
    return input_list

In [3]:
qbraid.get_devices()

Software Vendor    Device Provider    Device Name
-----------------  -----------------  -----------------------------
AWS                AWS                simulator_statevector
AWS                AWS                simulator_densitymatrix
AWS                AWS                simulator_tensornetwork
AWS                AWS                local_simulator_default
AWS                D-Wave             DW_2000Q_6
AWS                D-Wave             Advantage_system1
AWS                IonQ               ionQdevice
AWS                Rigetti            Aspen-9
Google             AQT                AQT
Google             Google             local_simulator_default
Google             Google             local_simulator_densitymatrix
IBM                IBMQ               armonk
IBM                IBMQ               belem
IBM                IBMQ               bogota
IBM                IBMQ               lima
IBM                IBMQ               manila
IBM                IBMQ               quit

In [4]:
ibm_wrapper = qbraid.device_wrapper("simulator_statevector", "IBMQ", vendor="IBM")

In [5]:
aws_wrapper = qbraid.device_wrapper("simulator_statevector", "AWS", vendor="AWS")

In [8]:
from qbraid.devices._utils import get_config

In [9]:
get_config("verify", "IBM")

'True'

In [10]:
from qiskit import IBMQ

In [11]:
IBMQ.active_account()

{'token': '7d6890d6f76e71f5c15ec68deedc2f030b192307c4baee6db8371a1634f9391789225f9881bec49c0be374473f29ee58c48168894b700476525689f020fda390',
 'url': 'https://auth.quantum-computing.ibm.com/api'}

In [12]:
IBMQ.disable_account()

In [14]:
IBMQ.active_account() is None

True

In [15]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [16]:
IBMQ.active_account()

{'token': '7d6890d6f76e71f5c15ec68deedc2f030b192307c4baee6db8371a1634f9391789225f9881bec49c0be374473f29ee58c48168894b700476525689f020fda390',
 'url': 'https://auth.quantum-computing.ibm.com/api'}

In [17]:
IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-08-10 15:24:02,549: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>